# Obteniendo el TOTAL de vacunados y fallecidos por COVID-19 por cada semana epidemiológica de todo el Perú

En este notebook se busca obtener un dataset con el total de **fallecidos** y vacunados **(dosis 1,2 y 3)** por cada **año** y **semana epidemiológica** de todo el Perú. 

Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

## 0. Cargar librerías

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from epiweeks import Week

import functions as fn

## 1. Cargar direcciones de RawData

El dataset de vacunados **'TB_VACUNACION_COVID19.csv'** no está incluido en el repositorio de GitHub debido a su excesivo tamaño (~2gb). Sin embargo se puede descargar directamente desde la página de Datos Abiertos de COVID-19 de MINSA, [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion) y colocarlo en la dirección RawData.

In [2]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

ubigeo_url = 'RawData/TB_UBIGEOS.csv'
vaccenter_url = 'RawData/TB_EESS.csv'

## 2. Procesar el dataset de fallecidos

El dataset de fallecidos será procesado con la librería **Pandas**.

### 2.1. Leer y procesar el dataset

Como se sabe que cada fila representa a una persona, solamente se toma la columna **'fecha de fallecimiento'**.

Se añaden 3 columnas. El año, la semana epidemiológica correspondiente y un contador para contar cada caso de fallecimiento por COVID-19. Se elimina la columna **'fecha_fallecimiento'**.

In [15]:
# Leer solamente la columna fecha de fallecimiento
fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO', 'EDAD_DECLARADA']
df_fal = pd.read_csv(fal_url, sep = ';', usecols = fal_col, dtype = {'FECHA_FALLECIMIENTO':'int32',
                                                                     'EDAD_DECLARADA': 'int8'})
del fal_col

# Transformar a formato fecha
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

# Agregamos el año y semana epidemilógica de cada fallecido
fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO') 
                             
df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8'})

del df_fal['FECHA_FALLECIMIENTO']
df_fal.head()

df_fal.head()

,EDAD_DECLARADA,DEPARTAMENTO,year,epi_week
0,41,LIMA,2021,16
1,31,CALLAO,2021,23
2,27,LIMA,2021,27
3,62,JUNIN,2021,20
4,74,LIMA,2021,20


In [4]:
# Data type
df_fal.dtypes

EDAD_DECLARADA      int8
DEPARTAMENTO      object
year               int16
epi_week            int8
dtype: object

Se cuenta cada caso de la variable **'fallecidos'** respecto al año y semana epidemiológica. 

In [5]:
# Devuelve un dataframe con el total de FALLECIDOS por año y semana epidemiológica
epi_fal = df_fal.groupby(['year', 'epi_week']).count()
epi_fal = epi_fal.rename(columns = {'DEPARTAMENTO':'total_fallecidos'})
del epi_fal['EDAD_DECLARADA']

epi_fal.head()

total_fallecidos
year epi_week                  
2020 10                       3
     11                       3
     12                      36
     13                      64
     14                     226

Se obtiene un dataframe **'epi_fal'** con el total de fallecidos por año y semana epidemiológica de todo el Perú.

## 3. Procesar el dataset de vacunados

### 3.1. Leer y procesar el dataset

Debido al tamaño del dataset de vacunados y para preservar recursos, no se procesará la información usando **Pandas**. En su lugar se utilizará el paquete **Dask**. La variable no será un **pandas dataframe** si no un **dask dataframe** el cual está separado en n particiones de tipo **pandas dataframe**.

Se añaden 2 columnas. El año y la semana epidemiológica correspondiente para cada vacunado contra COVID-19. Se elimina la columna **'fecha_vacunacion'**.

In [6]:
# Leemos solo la columna fecha vacunación y dosis del dataset de vacunados usando dask
vac_col = ['fecha_vacunacion','dosis']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8'})
del vac_col

# Personas con 3 o menos dosis
df_vac = df_vac[df_vac['dosis'] <= 3]

# Convertimos a formato fecha la columna 'fecha_vacunacion'
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')

df_vac.head()

,dosis,year,epi_week
0,2,2021,30
1,1,2021,35
2,1,2021,22
3,2,2021,39
4,1,2021,39


### 3.3. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [7]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['dosis']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [8]:
vac_dose = crosstab4dask(df_vac)
vac_dose.head()

dosis               1      2  3
year epi_week                  
2021 6         105762      1  0
     7          92346      2  0
     8         123974     19  0
     9          35270  93133  0
     10         74446  81336  0

Se obtiene un dataset **'vac_dose'** con el número de dosis aplicada por año y semana epidemiológica de todo el Perú.

## 4. Unir los dataframes de fallecidos y dosis de vacuna por año y semana epidemiológica

In [9]:
# Unimos ambos dataframes por semana epidemiológica
epiweeks = pd.concat([epi_fal, vac_dose], axis=1)  

epiweeks = epiweeks.fillna(value = 0) # Rellenamos vacíos o Nan values con 0
del epi_fal, vac_dose, df_vac

# Cambiamos a int ya que existen Nan values y cambia a float64 automáticamente
epiweeks = epiweeks.astype('int64')

# Rename columns
epiweeks.rename(columns={1: 'total_dosis_1', 2: 'total_dosis_2', 3: 'total_dosis_3'}, inplace=True)
                                    
epiweeks.head()

total_fallecidos  total_dosis_1  total_dosis_2  total_dosis_3
year epi_week                                                               
2020 10                       3              0              0              0
     11                       3              0              0              0
     12                      36              0              0              0
     13                      64              0              0              0
     14                     226              0              0              0

# Obteniendo el número de muertes confirmadas y vacunados contra COVID-19 por semana epidemiológica de cada uno de los 24 departamentos del Perú 

### 2.3. Realizar una crosstab de fallecidos por COVID-19 por departamento de Perú, para cada año y semana epidemiológica

Esto se lo realiza para obtener el TOTAL de fallecidos por cada departamento y semana epidemiológica

In [10]:
falxdep = pd.crosstab(index=[df_fal['year'], df_fal['epi_week']],
                      columns=[df_fal['DEPARTAMENTO']],
                      margins = False)

falxdep = falxdep.add_suffix('_fal')

falxdep.head()

DEPARTAMENTO   AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                   0           0             0             0   
     11                   0           0             0             0   
     12                   0           0             0             0   
     13                   0           4             0             0   
     14                   0           0             0            15   

DEPARTAMENTO   AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                   0              0           0          0   
     11                   0              0           0          0   
     12                   1              0           0          1   
     13                   0              0           2          3   
     14                   0              2          15          2   

DEPARTAMENTO   HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_fal  \
year epi_week                                 ...               
2020 10                       0            0  ...           1   
     11                       0            0  ...           1   
     12                       0            0  ...           3   
     13                       0            0  ...           6   
     14                       0            0  ...           9   

DEPARTAMENTO   MADRE DE DIOS_fal  MOQUEGUA_fal  PASCO_fal  PIURA_fal  \
year epi_week                                                          
2020 10                        0             0          0          0   
     11                        0             0          0          1   
     12                        0             0          0          1   
     13                        1             0          0          2   
     14                        0             0          0          9   

DEPARTAMENTO   PUNO_fal  SAN MARTIN_fal  TACNA_fal  TUMBES_fal  UCAYALI_fal  
year epi_week                                                                
2020 10               0               2          0           0            0  
     11               0               0          0           0            0  
     12               0               0          0           0            2  
     13               1               0          0           0            3  
     14               0               3          1           3           11  

[5 rows x 25 columns]

In [16]:
fn.variable_edad(df_fal, 'EDAD_DECLARADA')
del df_fal['DEPARTAMENTO']

df_fal.head()

,year,epi_week,edad_cat
0,2021,16,3
1,2021,23,2
2,2021,27,1
3,2021,20,5
4,2021,20,6


In [17]:
df_fal_edad = pd.crosstab(index = [df_fal['year'], df_fal['epi_week']], columns= df_fal['edad_cat'])
df_fal_edad = df_fal_edad.rename({0: '0_17_fal', 1:'18_29_fal', 2: '30_39_fal', 3:'40_49_fal', 4: '50_59_fal',
                                  5:'60_69_fal', 6:'70_79_fal', 7:'80_more_fal'}, axis=1)
df_fal_edad.head()

edad_cat       0_17_fal  18_29_fal  30_39_fal  40_49_fal  50_59_fal  \
year epi_week                                                         
2020 10               0          1          0          0          0   
     11               2          0          0          0          0   
     12               0          2          3          6          6   
     13               5          3          4          6          8   
     14               3          5         13         12         47   

edad_cat       60_69_fal  70_79_fal  80_more_fal  
year epi_week                                     
2020 10                1          0            1  
     11                0          1            0  
     12                5          8            6  
     13               12         16           10  
     14               47         51           48

Ahora tenemos el número de fallecidos por cada departamento y semana epidemiológica

## 3. Procesar el dataset de vacunados

No existe una manera directa de calcular el total de personas completamente vacunadas por departamento. Para lograrlo se prevee lo siguiente:

- El dataset de vacunados (RawData/TB_VACUNACION_COVID19.csv) solo proporciona información sobre el centro de vacunación llamado **'id_centro_vacunacion'**. NO el departamento.

- El dataset de los centros de vacunación [(RawData/TB_CENTRO_VACUNACION.csv)]() se puede utilizar para hacer **"match"** del **'id_centro_vacunacion'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica de 0 a 1894 que representa a cada distrito.

- Finalmente, con el dataset de UBIGEO [(RawData/TB_UBIGEOS.csv)]() es posible reemplazar a cada **'id_ubigeo'** con el departamento respectivo.

In [18]:
# Se lee otro dataset porque es más eficiente que separar y crear otro
vac_col = ['fecha_vacunacion','dosis', 'id_eess', 'edad']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8','id_eess':'int32', 'edad': 'int8'})
del vac_col

# Dejar solamente a personas con 2 dosis (será el dataset para el dataset en departamentos)
df_vac = df_vac[df_vac['dosis'] == 2]
del df_vac['dosis']

# Convertimos a formato fecha la columna 'fecha_vacunacion'
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')


def edad_cat(df, col):
    """Función para categorizar edad"""

    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]

    return np.select(conditions, choices, default=np.nan).astype('int8')

df_vac['edad_cat'] = df_vac.map_partitions(edad_cat, 'edad')
del df_vac['edad']

df_vac.head()

,id_eess,year,epi_week,edad_cat
0,21367,2021,30,4
3,16954,2021,39,2
7,23890,2021,48,0
10,8147,2021,27,4
11,20361,2021,29,4


In [19]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['edad_cat']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [20]:
df_vac_edad = crosstab4dask(df_vac)
df_vac_edad = df_vac_edad.rename({0: '0_17_vac', 1:'18_29_vac', 2: '30_39_vac', 3:'40_49_vac', 4: '50_59_vac',
                                  5:'60_69_vac', 6:'70_79_vac', 7:'80_more_vac'}, axis=1)
df_vac_edad.head()

edad_cat       0_17_vac  18_29_vac  30_39_vac  40_49_vac  50_59_vac  \
year epi_week                                                         
2021 6                0          0          0          1          0   
     7                0          0          1          1          0   
     8                0          4          6          2          4   
     9                0      11032      33048      25038      17121   
     10               3      13337      25387      19583      13842   

edad_cat       60_69_vac  70_79_vac  80_more_vac  
year epi_week                                     
2021 6                 0          0            0  
     7                 0          0            0  
     8                 2          1            0  
     9              6816         75            3  
     10             8994        155           35

### 3.3. Procesar los datasets de **UBIGEO** y **CENTROS DE VACUNACIÓN**.

#### 3.3.1. Leer y procesar los datasets

Para el dataset de UBIGEO solo se necesita las columnas 'id_ubigeo' y 'departamento'. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas 'id_centro de vacunación' e 'id_ubigeo'.

In [21]:
# Leemos los datasets de ubigeo y centros de vacunación
ubigeo = pd.read_csv(ubigeo_url, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(vaccenter_url, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

vaccenter.head()

,id_eess,departamento
0,17241,LORETO
1,14145,LORETO
2,7083,LORETO
3,549,LORETO
4,3207,LORETO


Ahora con el nuevo dataset **(vaccenter)** es posible encontrar el departamento correspondiente a 'id_centro de vacunacion' del dataset de vacunados **(df_vac)**.

#### 3.3.3. Encontrar el departamento correspondiente de todas las personas vacunadas

In [22]:
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')
del df_vac['id_eess']
del vaccenter

df_vac.head()

,year,epi_week,edad_cat,departamento
0,2021,30,4,LIMA
1,2021,39,2,CUSCO
2,2021,48,0,LIMA
3,2021,27,4,JUNIN
4,2021,29,4,LIMA


### 3.4. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [23]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['departamento']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [24]:
vac_dose = crosstab4dask(df_vac)
del df_vac

vac_dose = vac_dose.add_suffix('_vac')
vac_dose.head()

departamento   AMAZONAS_vac  ANCASH_vac  APURIMAC_vac  AREQUIPA_vac  \
year epi_week                                                         
2021 6                    0           0             0             0   
     7                    0           0             0             0   
     8                    0           2             0             0   
     9                  887        1975          2497          3763   
     10                 529        1083           834          4584   

departamento   AYACUCHO_vac  CAJAMARCA_vac  CALLAO_vac  CUSCO_vac  \
year epi_week                                                       
2021 6                    0              0           0          0   
     7                    0              0           0          1   
     8                    0              0           0          0   
     9                 2362           2726        4640       3205   
     10                2804            939        5449       3905   

departamento   HUANCAVELICA_vac  HUANUCO_vac  ...  LORETO_vac  \
year epi_week                                 ...               
2021 6                        0            0  ...           0   
     7                        0            0  ...           0   
     8                        0            1  ...           1   
     9                      887         2032  ...        1754   
     10                    1620          181  ...        1263   

departamento   MADRE DE DIOS_vac  MOQUEGUA_vac  PASCO_vac  PIURA_vac  \
year epi_week                                                          
2021 6                         0             0          0          0   
     7                         0             0          0          0   
     8                         0             0          3          0   
     9                       373          1006        816       3833   
     10                      168          1373        561       2103   

departamento   PUNO_vac  SAN MARTIN_vac  TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                                                
2021 6                0               0          0           0            0  
     7                0               0          0           0            0  
     8                0               0          0           0            0  
     9             1547            2929       1207         731          456  
     10            2356            4408       1665         944         1230  

[5 rows x 25 columns]

In [25]:
falvac_dep = pd.concat([falxdep, vac_dose], axis = 1).fillna(0)
falvac_dep.head()

AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                 0.0         0.0           0.0           0.0   
     11                 0.0         0.0           0.0           0.0   
     12                 0.0         0.0           0.0           0.0   
     13                 0.0         4.0           0.0           0.0   
     14                 0.0         0.0           0.0          15.0   

               AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                 0.0            0.0         0.0        0.0   
     11                 0.0            0.0         0.0        0.0   
     12                 1.0            0.0         0.0        1.0   
     13                 0.0            0.0         2.0        3.0   
     14                 0.0            2.0        15.0        2.0   

               HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_vac  \
year epi_week                                 ...               
2020 10                     0.0          0.0  ...         0.0   
     11                     0.0          0.0  ...         0.0   
     12                     0.0          0.0  ...         0.0   
     13                     0.0          0.0  ...         0.0   
     14                     0.0          0.0  ...         0.0   

               MADRE DE DIOS_vac  MOQUEGUA_vac  PASCO_vac  PIURA_vac  \
year epi_week                                                          
2020 10                      0.0           0.0        0.0        0.0   
     11                      0.0           0.0        0.0        0.0   
     12                      0.0           0.0        0.0        0.0   
     13                      0.0           0.0        0.0        0.0   
     14                      0.0           0.0        0.0        0.0   

               PUNO_vac  SAN MARTIN_vac  TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                                                
2020 10             0.0             0.0        0.0         0.0          0.0  
     11             0.0             0.0        0.0         0.0          0.0  
     12             0.0             0.0        0.0         0.0          0.0  
     13             0.0             0.0        0.0         0.0          0.0  
     14             0.0             0.0        0.0         0.0          0.0  

[5 rows x 50 columns]

# 6. Unir ambos dataframes

In [26]:
epiweeks_final = pd.concat([epiweeks, df_fal_edad, df_vac_edad, falvac_dep, ], axis = 1).fillna(0).astype('int64')
epiweeks_final.head()

total_fallecidos  total_dosis_1  total_dosis_2  total_dosis_3  \
year epi_week                                                                  
2020 10                       3              0              0              0   
     11                       3              0              0              0   
     12                      36              0              0              0   
     13                      64              0              0              0   
     14                     226              0              0              0   

               0_17_fal  18_29_fal  30_39_fal  40_49_fal  50_59_fal  \
year epi_week                                                         
2020 10               0          1          0          0          0   
     11               2          0          0          0          0   
     12               0          2          3          6          6   
     13               5          3          4          6          8   
     14               3          5         13         12         47   

               60_69_fal  ...  LORETO_vac  MADRE DE DIOS_vac  MOQUEGUA_vac  \
year epi_week             ...                                                
2020 10                1  ...           0                  0             0   
     11                0  ...           0                  0             0   
     12                5  ...           0                  0             0   
     13               12  ...           0                  0             0   
     14               47  ...           0                  0             0   

               PASCO_vac  PIURA_vac  PUNO_vac  SAN MARTIN_vac  TACNA_vac  \
year epi_week                                                              
2020 10                0          0         0               0          0   
     11                0          0         0               0          0   
     12                0          0         0               0          0   
     13                0          0         0               0          0   
     14                0          0         0               0          0   

               TUMBES_vac  UCAYALI_vac  
year epi_week                           
2020 10                 0            0  
     11                 0            0  
     12                 0            0  
     13                 0            0  
     14                 0            0  

[5 rows x 70 columns]

## 6. Guardar el dataset

In [27]:
epiweeks_final.to_csv('Data/DP1_vacunados_y_fallecidos_x_semanaEpi.csv')

Finalmente se guarda el dataset final con el ['Total de vacunados y fallecidos por año y semana epidemiológica'](https://github.com/xxotto/covid19-peru/blob/main/Data/TOTAL_vacunados_y_fallecidos_x_semanaEpi.csv) de todo el Perú.

Disponible en el directorio [Data](https://github.com/xxotto/covid19-peru/tree/main/Data).